In [5]:
import tensorflow as tf;
import numpy as np;
import matplotlib.pyplot as plt;
print(tf.__version__);
import pandas as pd;

from sklearn.metrics import mean_squared_error;

SyntaxError: invalid syntax (<ipython-input-5-4d5d0ecf695b>, line 1)

In [6]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [7]:
get_available_gpus()

[]

In [2]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

In [3]:
# tf.debugging.set_log_device_placement(True)

In [4]:
multi_data = pd.read_csv('../data/zri_multifamily_v2.csv');
zip_ids = multi_data["zip"].unique();

In [5]:
MONTHS = 72;
SPLIT = 60; # 2014-2018: training, 2019: testing.
BATCH_SIZE = 4;
WINDOW_SIZE = 6;

TEST_LENGTH = MONTHS - SPLIT;

In [6]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [7]:
def NN_model(dataset):
    tf.keras.backend.clear_session()
    # dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

    model = tf.keras.models.Sequential([
        tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                          input_shape=[None]),
    #   tf.keras.layers.Conv1D(filters=16, kernel_size=3,
    #                       strides=1, padding="causal",
    #                       activation="relu",
    #                       input_shape=[None, 1]),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
        #  tf.keras.layers.SimpleRNN(16, return_sequences=True),
        #  tf.keras.layers.SimpleRNN(16, return_sequences=True),
    #   tf.keras.layers.Dense(16, activation="relu"),
    #   tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1),
        tf.keras.layers.Lambda(lambda x: x * 2.0)
    ])

    optimizer = tf.keras.optimizers.SGD(learning_rate=3e-4, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(),
                  optimizer="adam",
                  metrics=["mae", "mse"])
    history = model.fit(dataset, epochs=500, verbose = 0);
    return model;

In [8]:
def NN_forecast(model, single_city_series):
    forecast = []
    results = []
    for time in range(len(single_city_series) - WINDOW_SIZE):
        forecast.append(model.predict(single_city_series[time:time + WINDOW_SIZE][np.newaxis]))

    #print(forecast)

    # forecast = forecast[SPLIT - WINDOW_SIZE:]
    results = np.array(forecast)[:, 0, 0]
    actual = single_city_series[WINDOW_SIZE:]
    time_actual = range(WINDOW_SIZE, len(single_city_series));

    pure_forecast = list(single_city_series[SPLIT - WINDOW_SIZE: SPLIT]);
    for time in range(SPLIT, len(single_city_series)):
        # print(model.predict(pure_forecast[-WINDOW_SIZE:][np.newaxis]))
        pure_forecast.append(np.array(model.predict(np.array(pure_forecast[-WINDOW_SIZE:])[np.newaxis]))[0][0][0])
    pure_forecast = np.array(pure_forecast[WINDOW_SIZE:]);

    return results, actual, pure_forecast;

In [36]:
@tf.autograph.experimental.do_not_convert
def NN_test(ZONE):
    '''
    Input: ZONE
    Output: the RMSE of a NN model on the predicted train, partially predicted test, and complete predicted test.
    '''
    # Collection of data
    single_city_data = multi_data[multi_data["zip"] == ZONE];
    single_city_series = np.array(single_city_data["zri"]);
    
    # Standardization
    single_city_series_mean = single_city_series.mean();
    single_city_series_std = single_city_series.std();

    single_city_series = (single_city_series - single_city_series_mean)/\
    single_city_series_std;
    
    # Train test split
    single_city_train = single_city_series[:SPLIT];
    single_city_test = single_city_series[SPLIT:];
    
    # Window the training set to make input of the NN
    dataset = windowed_dataset(single_city_train, WINDOW_SIZE, BATCH_SIZE, 60);
    model = NN_model(dataset);
    
    time_train = list(range(SPLIT));
    time_test = list(range(SPLIT, len(single_city_series)));
    
    # Forecasting
    results, actual, pure_forecast = NN_forecast(model, single_city_series);
    
    # Compute MSE
    MSE_train = mean_squared_error(actual[:-TEST_LENGTH], results[:-TEST_LENGTH])**0.5 * single_city_series_std;
    MSE_test = mean_squared_error(actual[-TEST_LENGTH:], results[-TEST_LENGTH:])**0.5 * single_city_series_std;
    MSE_pure = mean_squared_error(actual[-TEST_LENGTH:], pure_forecast[-TEST_LENGTH:])**0.5 * single_city_series_std;

    return MSE_train, MSE_test, MSE_pure;

In [10]:
# ZONE = 11050;
# with tf.device('/GPU:1'):
#     %time M_train, M_test, M_pure = NN_test(11050);

In [38]:
# score_dict = {"zip":[], "RMSE_train":[], "RMSE_test":[], "RMSE_pure":[]};
for ZONE in zip_ids[80:100]:
    # @tf.autograph.experimental.do_not_convert
    %time M_train, M_test, M_pure = NN_test(ZONE);
    print(ZONE, M_train, M_test, M_pure);
    
    score_dict["zip"].append(ZONE);
    score_dict["RMSE_train"].append(M_train);
    score_dict["RMSE_test"].append(M_test);
    score_dict["RMSE_pure"].append(M_pure);
    

Wall time: 40.1 s
2171 10.098839899350098 23.781687888690282 34.82469171474734
Wall time: 38.9 s
2176 13.681975283783459 86.6782220558549 137.13205430032463
Wall time: 35.6 s
2184 13.700445996724275 26.578455158878963 98.08577096887959
Wall time: 36 s
2210 15.677227673503474 235.09357682226795 198.23610569231573
Wall time: 34.7 s
2215 15.923420389722926 101.9758159448398 136.57908480573835
Wall time: 35.4 s
2301 18.808529199983884 31.683275605945433 75.42045276220445
Wall time: 36.3 s
2302 9.00731979227754 27.275937924334446 39.09364047206697
Wall time: 35.8 s
2360 11.079577758292107 32.221725275782426 99.42050290515948
Wall time: 34.8 s
2445 10.881034124589405 51.6293691279428 119.64962665956406
Wall time: 35.8 s
2446 9.687786769485061 84.40144795839784 72.97755462930269
Wall time: 37.2 s
2451 8.842505460243453 48.91230166895163 115.26376913735672
Wall time: 35.4 s
2453 11.11757317136014 44.12391189157893 83.31043081050431
Wall time: 35.5 s
2458 6.696918920517107 44.792637630020295 89

In [39]:
print(len(score_dict["RMSE_pure"]), sum(score_dict["RMSE_pure"])/len(score_dict["RMSE_pure"]));

100 90.77862133503896


In [40]:
test = pd.DataFrame(score_dict)

In [35]:
test.head(50)

,zip,RMSE_train,RMSE_test,RMSE_pure
0,1013,6.547411,13.259700,21.821001
1,1020,6.721370,28.159768,39.378511
2,1040,6.190981,15.454253,42.897026
3,1085,6.707341,28.105983,34.235139
4,1104,9.726347,20.828578,28.879755
5,1108,6.453867,24.347008,27.943128
6,1109,7.874041,18.629411,36.359158
7,1420,6.399677,50.914301,80.277967
8,1440,6.589762,175.134164,151.288363
9,1453,7.249337,13.032851,15.845272


In [33]:
list((multi_data["zip"].unique()))[:30]

[1013,
 1020,
 1040,
 1085,
 1104,
 1108,
 1109,
 1420,
 1440,
 1453,
 1545,
 1550,
 1602,
 1603,
 1604,
 1605,
 1610,
 1702,
 1752,
 1757,
 1760,
 1801,
 1830,
 1832,
 1841,
 1843,
 1844,
 1845,
 1850,
 1851]

In [34]:
multi_data.head()

,zip,City,State,Metro,CountyName,year-month,zri
0,1013,Chicopee,MA,Springfield,Hampden County,2014-01,928.0
1,1013,Chicopee,MA,Springfield,Hampden County,2014-02,931.0
2,1013,Chicopee,MA,Springfield,Hampden County,2014-03,934.0
3,1013,Chicopee,MA,Springfield,Hampden County,2014-04,929.0
4,1013,Chicopee,MA,Springfield,Hampden County,2014-05,929.0
